## Shell Contact Example

In [1]:
# !pip install debugpy

In [2]:
import numpy as np

import dismech


geom = dismech.GeomParams(rod_r0=0.005,
                          shell_h=0.001)

material = dismech.Material(density=2000,
                            youngs_rod=0,
                            youngs_shell=2e10,
                            poisson_rod=0,
                            poisson_shell=0.5)

shell_contact_sim = dismech.SimParams(static_sim=False,
                                  two_d_sim=False,   # no twisting
                                  use_mid_edge=False,
                                  use_line_search=True,
                                  show_floor=False,
                                  log_data=True,
                                  log_step=1,
                                  dt=1e-4,
                                  max_iter=120,
                                  total_time=0.5,
                                  plot_step=1,
                                  tol=1e-4,
                                  ftol=1e-4,
                                  dtol=1e-2)

env = dismech.Environment()
env.add_force('gravity', g=np.array([0.0, 0.0, -9.81]))
# env.add_force('floorContact', ground_z=-1e-3, stiffness=20, delta=1e-4, h=1e-3)
env.add_force('selfContact', delta=1e-4, h=1e-3, kc=10)
env.add_force('damping', eta = 0.1)
#env.set_static()   

geo = dismech.Geometry.from_txt('2random_mesh_20.txt')
# geo = dismech.Geometry.from_txt('2random_mesh_8.txt')
# geo = dismech.Geometry.from_txt('input_shell_more_curved_scaled.txt')

robot = dismech.SoftRobot(geom, material, geo, shell_contact_sim, env)

# fixed_points = np.array(np.where(
#     (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 0] <= 0.01) &
#     (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 2] == 0.0)
# )[0])
fixed_points = np.array(np.where(
    (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 2] < 0.001) # (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 2] < 0.001)
)[0])

# fixed_points = np.array(np.where(
#     (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 0] <= 0.455) & 
#     (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 0] >= 0.445) # (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 2] < 0.001)
# )[0])


robot = robot.fix_nodes(fixed_points)

# u = robot.state.u
# u[11] = -0.5
# u[14] = -0.5
# u[17] = -0.5

### Time Stepping

As we are performing a static simulation, we must change gravity for each time step.

In [3]:
stepper = dismech.ImplicitEulerTimeStepper(robot)

robots = stepper.simulate()

qs = np.stack([robot.state.q for robot in robots])

delta: 0.1
h: 1.0
K1: 150.0
scale:  1000.0
kc:  10
upper limit for quadratic: 1.9
upper limit for smooth: 2.1
48388.1316995642
12472.002606346
32.76684323053352
48388.466248495744
12472.043688696345
32.76639950803117
48388.46614950143
12472.043679616852
32.76639960735984
48388.46614952292
12472.043679618982
32.76639960708144
48388.466149523454
12472.043679619019
32.766399606997446
48388.466149524356
12472.043679619212
32.766399606818354
48388.46614952405
12472.043679618768
32.76639960699895
16.399743547560483
8.204031038850157
4.103058270103256
current_time:  0.0001
2.172708133893261
current_time:  0.0002
1.2897235626864387
current_time:  0.00030000000000000003
1.6676617403198888
current_time:  0.0004
2.0541115506251653
current_time:  0.0005
2.4436817933741883
current_time:  0.0006000000000000001
2.8347527188109027
current_time:  0.0007
3.226493737036918
0.00012295037833736906
current_time:  0.0008
3.6184201289844733
9.129925265877602e-05
current_time:  0.0009000000000000001
4.01019667

ValueError: Iteration limit 120 reached before convergence

In [ ]:
t = np.arange(robot.sim_params.total_time, step=robot.sim_params.dt)
options = dismech.AnimationOptions(title='Shell contact p2p', plot_step=100)

fig = dismech.get_interactive_animation_plotly(robot, t, qs, options)
fig.show()

In [5]:
# ani = dismech.get_animation(robot, t, qs, options)
# ani.save('5e-5.gif')